In [ ]:
# -*- coding: utf-8 -*-
# Scripts/test_bus_boarding.py
# 单个 agent 走到最近车站并上车的演示脚本

import sys
import os
import json
import math
import copy

from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QTimer

# 路径设置
SIMWORLD_DIR      = r"D:\Projects\Food-Delivery-Bench\SimWorld"
LLM_DELIVERY_DIR  = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery"
sys.path.insert(0, SIMWORLD_DIR); sys.path.insert(0, LLM_DELIVERY_DIR)

from Base.Map import Map
from Base.Order import OrderManager
from Base.DeliveryMan import DeliveryMan, TransportMode, DMAction, DMActionKind
from Base.Store import StoreManager
from utils.map_observer import MapObserver
from Base.Timer import VirtualClock
from Base.Comms import init_comms

# 公交系统
from Base.Bus import Bus, BusRoute, BusStop
from Base.BusManager import BusManager

from Communicator import Communicator

ROADS_JSON        = r"D:\Projects\Food-Delivery-Bench\Test_Data\test\roads.json"
WORLD_JSON        = r"D:\Projects\Food-Delivery-Bench\Test_Data\test\progen_world_enriched.json"
STORE_ITEMS_JSON  = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery\input\store_items.json"
FOOD_JSON         = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery\input\food.json"
CONFIG_JSON       = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery\input\config.json"
SPECIAL_NOTES_JSON = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery\input\special_notes.json"

def _load_world_nodes(path: str):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f).get("nodes", [])

def _load_cfg(path: str) -> dict:
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f) or {}
        return data
    except FileNotFoundError:
        raise RuntimeError(f"Config file not found: {path}")
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Config JSON parse error in {path}: {e}")
    

def main():
    app = QApplication(sys.argv)

    # --- 地图/订单/商店 ---
    m = Map(); m.import_roads(ROADS_JSON); m.import_pois(WORLD_JSON)
    nodes = _load_world_nodes(WORLD_JSON)

    # 读取 food.json
    with open(FOOD_JSON, "r", encoding="utf-8") as f:
        food_data = json.load(f) or {}
    menu_items = food_data.get("items", [])

    cfg = _load_cfg(CONFIG_JSON)

    # Clock
    clock = VirtualClock(time_scale=1.0)

    # Comms
    comms = init_comms(clock=clock)

    om = OrderManager(capacity=10, menu=menu_items, clock=clock)
    om.fill_pool(m, nodes)

    sm = StoreManager(); sm.load_items(STORE_ITEMS_JSON)

    # --- 公交系统初始化 ---
    bus_manager = BusManager(clock=clock, waiting_time_s=cfg.get("bus", {}).get("waiting_time_s", 10.0), speed_cm_s=cfg.get("bus", {}).get("speed_cm_s", 1200.0))
    
    # 从世界数据加载公交路线
    with open(WORLD_JSON, "r", encoding="utf-8") as f:
        world_data = json.load(f)
    bus_manager.init_bus_system(world_data)

    # --- 单实例 UE 通信（9000口） ---
    communicator = Communicator(port=9000, ip='127.0.0.1', resolution=(640, 480))

    # --- Viewer ---
    v = MapObserver(title="Bus Boarding Test - Agent goes to nearest stop and boards bus", clock=clock)
    v.draw_map(m, WORLD_JSON, show_bus=True, show_docks=False,
               show_building_links=True, show_road_names=True, plain_mode="pudo")
    v.resize(1200, 900); v.show()
    v.attach_order_manager(om)
    v.attach_comms(comms)
    v.attach_bus_manager(bus_manager)  # 绑定公交管理器


    ax, ay = (-20000, -20000)
    dm = DeliveryMan("bus_agent", m, nodes, ax, ay, mode=TransportMode.WALK, clock=clock, cfg=copy.deepcopy(cfg))

    dm.bind_viewer(v)
    dm.set_order_manager(om)
    dm.set_store_manager(sm)
    dm.set_bus_manager(bus_manager)  # 绑定公交管理器
    dm.set_ue(communicator)  # 绑定 UE
    dm.bind_simworld()       # 在 UE 里 spawn
    dm.vlm_add_memory("wants to use public transport")
    dm.register_to_comms()

    # --- 同步屏障：等待 agent 在 UE 中初始化 ---
    ready = False

    def check_ready():
        nonlocal ready
        if ready:
            return
        rec = communicator.get_position_and_direction(str(dm.agent_id))
        tup = rec.get(str(dm.agent_id)) if rec else None
        if tup:  # 能拿到 loc+rot，说明 UE 中的 Actor 已经就绪
            ready = True
            ready_timer.stop()
            dm._log(f"Agent {dm.agent_id} initialized successfully at ({dm.x/100.0:.2f}m, {dm.y/100.0:.2f}m)")
            start_bus_boarding_demo()

    ready_timer = QTimer(v)
    ready_timer.setInterval(100)  # 10Hz 轮询
    ready_timer.timeout.connect(check_ready)
    ready_timer.start()

    def start_bus_boarding_demo():
        """开始公交上车演示"""
        print("\n=== 公交上车演示开始 ===")
        
        # 1. 查找最近的公交站点
        nearest_stop, distance = bus_manager.find_nearest_bus_stop(dm.x, dm.y)
        if nearest_stop:
            print(f"找到最近公交站: {nearest_stop.name or nearest_stop.id} 距离 {distance/100:.1f}m")
            
            # 2. 移动到公交站点
            print(f"Agent 正在前往公交站...")
            dm.enqueue_action(DMAction(DMActionKind.MOVE_TO, data={
                "tx": nearest_stop.x, 
                "ty": nearest_stop.y, 
                "use_route": True, 
                "snap_cm": 120.0
            }))
            
            # 3. 到达站点后等待公交车
            def wait_and_board_bus():
                print("Agent 已到达公交站，等待公交车...")
                
                # 检查是否有公交车在附近
                available_buses = []
                for bus_id, bus in bus_manager.buses.items():
                    if bus.is_at_stop() and math.hypot(bus.x - dm.x, bus.y - dm.y) <= 300.0:
                        available_buses.append(bus_id)
                
                if available_buses:
                    bus_id = available_buses[0]
                    print(f"发现公交车 {bus_id}，准备上车...")
                    
                    # 上车
                    dm._handle_board_bus(dm, DMAction(DMActionKind.BOARD_BUS, data={
                        "bus_id": bus_id,
                        "target_stop": "GEN_POI_BusStation_2_438"
                    }), False)
                else:
                    print("当前没有公交车在站，继续等待...")
                    # 5秒后再次检查
                    QTimer.singleShot(2000, wait_and_board_bus)
            
            # 延迟执行等待和上车（给移动一些时间）
            QTimer.singleShot(2000, wait_and_board_bus)
            
        else:
            print("未找到公交站点！")

    # --- 周期读取 UE 坐标 ---
    ue_timer = QTimer(v)
    ue_timer.setInterval(150)
    ue_timer.start()

    # --- 状态打印定时器 ---
    def print_status():
        print("\n=== 系统状态 ===")
        print(f"Agent: {dm.to_text()}")
        print("Buses:")
        for status in bus_manager.get_all_buses_status():
            print(f"  {status}")
        print("-" * 60)

    status_timer = QTimer(v)
    status_timer.setInterval(5000)  # 每5秒打印一次
    status_timer.timeout.connect(print_status)
    status_timer.start()

    # 启动agent
    # dm.kickstart()

    sys.exit(app.exec_())

if __name__ == "__main__":
    main()